In [1]:
import random
import math


def calculate_distance(city1, city2):
    """
    Calculates the Euclidean distance between two cities.
    """
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)


def calculate_distance_matrix(cities):
    """
    Generates a distance matrix from a list of cities and their coordinates.
    """
    n = len(cities)
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            distance = calculate_distance(cities[i], cities[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance
    return distance_matrix


def randomized_nearest_neighbor(cities, distance_matrix):
    """
    Generates an initial route using the randomized nearest neighbor algorithm.
    """
    n = len(cities)
    visited_cities = [False] * n
    route = [0] * n

    # Select a random starting city.
    current_city = random.randint(0, n-1)
    visited_cities[current_city] = True
    route[0] = current_city

    # Add the remaining cities to the route.
    for i in range(1, n):
        # Compute the distances from the last city added to all remaining cities.
        distances = [(distance_matrix[current_city][j], j)
                     for j in range(n) if not visited_cities[j]]
        # Select the city with the shortest distance to the last city and add it to the route.
        min_distance, nearest_city = min(distances)
        visited_cities[nearest_city] = True
        route[i] = nearest_city
        current_city = nearest_city

    return route


def calculate_total_distance(route, distance_matrix):
    """
    Calculates the total distance of a route given its distance matrix.
    """
    total_distance = 0
    n = len(route)
    for i in range(n-1):
        total_distance += distance_matrix[route[i]][route[i+1]]
    total_distance += distance_matrix[route[-1]][route[0]]
    return total_distance


def swap_cities(route, i, j):
    """
    Swaps two cities in a route.
    """
    new_route = route[:]
    new_route[i], new_route[j] = new_route[j], new_route[i]
    return new_route


def apply_2opt_swap(route, i, j):
    """
    Applies the 2-opt swap to a route.
    """
    new_route = route[:i] + route[i:j+1][::-1] + route[j+1:]
    return new_route


def run_tsp(cities, num_iterations):
    distance_matrix = calculate_distance_matrix(cities)

    # Generate an initial route.
    initial_route = list(range(len(cities)))
    random.shuffle(initial_route)
    initial_distance = calculate_total_distance(initial_route, distance_matrix)

    # Generate a route using the randomized nearest neighbor algorithm.
    rnna_route = randomized_nearest_neighbor(cities, distance_matrix)
    rnna_distance = calculate_total_distance(rnna_route, distance_matrix)

    # Apply the 2-opt algorithm to improve the route.
    current_best_route = rnna_route
    current_best_distance = rnna_distance

    for i in range(num_iterations):
        # Select two random cities in the current best route.
        city1, city2 = random.sample(range(len(initial_route)), 2)
        # Apply the 2-opt swap to the current best route.
        new_route = apply_2opt_swap(current_best_route, city1, city2)

        # Calculate the total distance of the new route.
        new_distance = calculate_total_distance(new_route, distance_matrix)

        # If the new route is shorter than the current best route, update the current best route.
        if new_distance < current_best_distance:
            current_best_route = new_route
            current_best_distance = new_distance

    print("Initial Route: ", initial_route)
    print("Initial Distance: ", initial_distance, "km")

    print("\nRandomized Nearest Neighbor Route: ", rnna_route)
    print("Randomized Nearest Neighbor Distance: ", rnna_distance, "km")
    print("Improved by RNNA: -", initial_distance - rnna_distance, "km | In percentage: ",
          (initial_distance - rnna_distance) / initial_distance * 100, "%")

    print("\nOptimized Route: ", current_best_route)
    print("Optimized Distance: ", current_best_distance, "km")
    print("Improved by 2-Opt Swap: -", rnna_distance - current_best_distance,
          "km | In percentage: ", (rnna_distance - current_best_distance) / rnna_distance * 100, "%")

In [2]:
# TEST CASE 1 - 15 cities; 1000 iterations
# Original proposed solution to the vaccine distribution problem
# Carl's note: I wanted to use the real location of the cities in Quezon, but due to time constraints and conflicting priorities, I used random coordinates. I wish to implement this in the future.

num_iterations = 1000
cities = [(110.0, 225.0), (161.0, 283.0), (188.0, 293.0), (264.0, 292.0),    (280.0, 231.0), (333.0, 198.0), (340.0, 149.0),
          (366.0, 129.0),    (382.0, 179.0), (380.0, 218.0), (394.0, 261.0), (413.0, 242.0),    (449.0, 269.0), (477.0, 246.0), (469.0, 205.0)]

run_tsp(cities, num_iterations)

Initial Route:  [12, 7, 10, 3, 6, 5, 0, 4, 13, 11, 2, 1, 8, 9, 14]
Initial Distance:  1999.0005498593391 km

Randomized Nearest Neighbor Route:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 13, 14]
Randomized Nearest Neighbor Distance:  1042.1970100478286 km
Improved by RNNA: - 956.8035398115105 km | In percentage:  47.86409587925509 %

Optimized Route:  [0, 1, 2, 3, 10, 12, 13, 14, 11, 9, 8, 7, 6, 5, 4]
Optimized Distance:  963.5770192510071 km
Improved by 2-Opt Swap: - 78.61999079682153 km | In percentage:  7.543678406179029 %


In [3]:
# TEST CASE 2 - 1000 cities; 10000 iterations
# Scaled up version of the original proposed solution to the vaccine distribution problem.
# Wider spread of cities.
# More iterations to improve the route.

num_iterations = 10000
cities = [(random.randint(0, 1000), random.randint(0, 1000))
          for i in range(1000)]

run_tsp(cities, num_iterations)


Initial Route:  [552, 560, 421, 554, 309, 232, 474, 629, 743, 889, 605, 457, 348, 515, 831, 101, 857, 717, 473, 329, 699, 644, 865, 365, 686, 749, 631, 736, 964, 370, 687, 921, 618, 823, 924, 187, 640, 449, 695, 264, 528, 688, 463, 521, 862, 302, 683, 801, 184, 761, 919, 214, 601, 125, 871, 4, 52, 484, 782, 358, 795, 361, 875, 415, 496, 81, 285, 690, 621, 933, 563, 939, 366, 328, 682, 436, 741, 350, 443, 482, 105, 284, 483, 844, 69, 895, 735, 171, 951, 414, 128, 323, 367, 460, 123, 887, 706, 470, 119, 700, 592, 622, 404, 952, 441, 158, 840, 635, 137, 852, 290, 74, 78, 308, 856, 6, 973, 669, 610, 573, 223, 70, 34, 65, 322, 708, 238, 539, 206, 796, 107, 968, 659, 932, 786, 923, 853, 480, 639, 422, 649, 319, 582, 900, 466, 642, 603, 169, 179, 245, 525, 304, 977, 40, 845, 30, 727, 901, 935, 31, 435, 918, 989, 448, 405, 711, 978, 241, 133, 239, 96, 748, 487, 331, 180, 668, 21, 506, 433, 118, 189, 100, 362, 444, 909, 142, 808, 42, 456, 253, 345, 833, 556, 172, 166, 826, 534, 306, 598, 514, 1